In [1]:
import numpy as np
import rasterio
from sklearn.preprocessing import StandardScaler
import pymc as pm
import aesara.tensor as at

ModuleNotFoundError: No module named 'distutils.msvccompiler'

In [2]:
# Load rasters
with rasterio.open("/Users/chrisperez/Desktop/stat288-finalproject/data/cuba_ndvi_2024.tif") as src:
    ndvi_data = src.read(1)

with rasterio.open("/Users/chrisperez/Desktop/stat288-finalproject/data/cuba_viirs_2024.tif") as src:
    viirs_data = src.read(1)

with rasterio.open("/Users/chrisperez/Desktop/stat288-finalproject/data/cuba_osm_road_density_2024.tif") as src:
    road_data = src.read(1)

In [4]:
# Flatten all three layers
ndvi_flat = ndvi_data.flatten()
viirs_flat = viirs_data.flatten()
road_flat = road_data.flatten()

# Mask: valid values only and remove water (low NDVI)
valid_mask = (ndvi_flat > 0.05) & np.isfinite(ndvi_flat) & np.isfinite(viirs_flat) & np.isfinite(road_flat)

# Stack into feature matrix (N x 3)
X = np.stack([ndvi_flat[valid_mask], viirs_flat[valid_mask], road_flat[valid_mask]], axis=1)

# Save shape for reverse mapping
grid_shape = ndvi_data.shape

# Standardize the proxies (z-score)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Optionally save indices for mapping back to image
pixel_indices = np.argwhere(valid_mask).flatten()

In [5]:
# Get the number of locations and proxies
N = X_scaled.shape[0]     # number of spatial locations
J = X_scaled.shape[1]     # number of proxies (should be 3)

# Transpose data to get shape (J, N), so each row is a proxy across all locations
x_obs = X_scaled.T

# Print shape check
print(f"x_obs shape (J x N): {x_obs.shape}")  # should be (3, N)

x_obs shape (J x N): (3, 179974)
